In [ ]:
#Check Python Version
import shutil
import sys
import os
sys.version


#Upgrade pip
%pip install --upgrade pip


#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [ ]:
import json
import tempfile
from typing import Any, Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Any, Dict, List, Optional, Type, Union


from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from hello import component

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """TODO: Function that parses input_dict, creates a directory from the split-names and tfrecord uris provided"""

        input_train_dir='./examples_imported/Split-train/'
        input_eval_dir='examples_imported/Split-eval/'
        split_names='["train","eval"]'

        fileio.copy(input_train_dir, artifact_utils.get_split_uri(output_example, "train")) 
        fileio.copy(input_eval_dir, artifact_utils.get_split_uri(output_example, "eval")) 
        output_example.split_names=split_names


def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  # User input dictionary with split-names and their resepctive uri to tfrecords
  tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }
  split_names=''
  counter=1

#   cwd = os.getcwd()
#   temp_dir = tempfile.TemporaryDirectory(dir = cwd)

#   print(temp_dir.name)

  # Destination directory for source
  destination_examples_artifact_uri = './examples'
  print(destination_examples_artifact_uri)

  if not os.path.exists(destination_examples_artifact_uri):
      os.mkdir(destination_examples_artifact_uri)


  # Create a folder in destination_examples_artifact_uri for every key name
  for key in tfrecords_dict:
        split_name_temp = "Split-"+key
        path = os.path.join(destination_examples_artifact_uri, split_name_temp)
        if not os.path.exists(path):
            os.mkdir(path)
            print(path)
        shutil.copy(tfrecords_dict[key], path)
        print(path)


  # Create split_names string
  for key in tfrecords_dict:
      if(counter==1):
          split_names+='["'+key+'","'
          counter=counter+1
      elif(counter==2):
          split_names+=key+'"]'
      else:
          split_names+=key+','
          counter=counter+1

  print(split_names)

  # CopyExampleGen execution process to replace Importer Node functionality
  examples_importer = tfx.dsl.Importer(
        source_uri=destination_examples_artifact_uri,
        artifact_type=tfx.types.standard_artifacts.Examples,
        # properties={"split_names": '["train","eval"]'}
        properties={"split_names": split_names}
        ).with_id('examples_importer')


#   output_example: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Examples]
                                                    
  copy_examples=CopyExampleGen(
      input_dict=tfrecords_dict,
    #   output_example=output_example
  )

  
#   hello_component = component.HelloComponent(
#       input_data=examples_importer.outputs['result'],
#       tfrecords_dict=tfrecords_dict
#     #   input_data=tfrecords_dict
#   )

  # Test downstream component
  statistics_gen = tfx.components.StatisticsGen(
     examples=copy_examples.outputs['examples'])

  # Following three components will be included in the pipeline.
  components = [
      # example_gen,
      copy_examples,
      examples_importer,
    #   hello_component,
      statistics_gen
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

In [ ]:
import re
'./examples/'
dict_output = {
	"name": "ValueError",
	"message": "Generic type `Parameter[T]` expects the single parameter T to be `int`, `float`, `str`, `bytes`, `bool` or JSON-compatible types (Dict[str, T], List[T]) (got typing.Dict instead).",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mValueError\u001b[0m                                Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_1797540/3420384479.py\u001b[0m in \u001b[0;36m<cell line: 1>\u001b[0;34m()\u001b[0m\n\u001b[1;32m      1\u001b[0m tfx.orchestration.LocalDagRunner().run(\n\u001b[0;32m----> 2\u001b[0;31m   _create_pipeline(\n\u001b[0m\u001b[1;32m      3\u001b[0m       \u001b[0mpipeline_name\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mPIPELINE_NAME\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      4\u001b[0m       \u001b[0mpipeline_root\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mPIPELINE_ROOT\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      5\u001b[0m       \u001b[0mdata_root\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mDATA_ROOT\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m/tmp/ipykernel_1797540/745406203.py\u001b[0m in \u001b[0;36m_create_pipeline\u001b[0;34m(pipeline_name, pipeline_root, data_root, metadata_path)\u001b[0m\n\u001b[1;32m     73\u001b[0m   \u001b[0;34m@\u001b[0m\u001b[0mcomponent\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     74\u001b[0m   def CopyExampleGen(\n\u001b[0;32m---> 75\u001b[0;31m         \u001b[0minput_dict\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mtfx\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdsl\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcomponents\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mParameter\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mDict\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     76\u001b[0m         \u001b[0moutput_example\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mtfx\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdsl\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcomponents\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mOutputArtifact\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mtfx\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mtypes\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mstandard_artifacts\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mExamples\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     77\u001b[0m       ) -> tfx.dsl.components.OutputDict():\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/component/experimental/annotations.py\u001b[0m in \u001b[0;36m__getitem__\u001b[0;34m(cls, params)\u001b[0m\n\u001b[1;32m     78\u001b[0m                   params: Type[Union[int, float, str, bytes, bool]]):\n\u001b[1;32m     79\u001b[0m     \u001b[0;34m\"\"\"Metaclass method allowing indexing class (`_PrimitiveTypeGeneric[T]`).\"\"\"\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 80\u001b[0;31m     \u001b[0;32mreturn\u001b[0m \u001b[0mcls\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_generic_getitem\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mparams\u001b[0m\u001b[0;34m)\u001b[0m  \u001b[0;31m# pytype: disable=attribute-error\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     81\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     82\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/component/experimental/annotations.py\u001b[0m in \u001b[0;36m_generic_getitem\u001b[0;34m(cls, params)\u001b[0m\n\u001b[1;32m    105\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    106\u001b[0m       \u001b[0mclass_name\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mcls\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m__name__\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 107\u001b[0;31m       raise ValueError(\n\u001b[0m\u001b[1;32m    108\u001b[0m           ('Generic type `%s[T]` expects the single parameter T to be '\n\u001b[1;32m    109\u001b[0m            \u001b[0;34m'`int`, `float`, `str`, `bytes`, `bool` or JSON-compatible types '\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mValueError\u001b[0m: Generic type `Parameter[T]` expects the single parameter T to be `int`, `float`, `str`, `bytes`, `bool` or JSON-compatible types (Dict[str, T], List[T]) (got typing.Dict instead)."
}
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)

In [ ]:
from typing import Any, Dict
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Any, Dict, List, Optional, Type, Union

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio


tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """TODO: Function that parses input_dict, creates a directory from the split-names and tfrecord uris provided"""

        list=[output_example]
        input_train_dir='./examples_imported/Split-train/data_tfrecord-00000-of-00001.gz'
        input_eval_dir='examples_imported/Split-eval/data_tfrecord-00000-of-00001.gz'
        split_names='["train","eval"]'
        print("Print output_example" + str(output_example))

        fileio.copy(input_train_dir, artifact_utils.get_split_uri(list, "train")) 
        fileio.copy(input_eval_dir, artifact_utils.get_split_uri(list, "eval")) 
        output_example.split_names=split_names


copy_examples=CopyExampleGen(input_dict=tfrecords_dict)

context=InteractiveContext()
context.run(copy_examples)